In [1]:
!pip3 install matplotlib pyss3 sklearn

     |████████████████████████████████| 2.0MB 2.8MB/s 


In [0]:
%matplotlib inline
from pyss3 import SS3
from pyss3.util import Dataset, Evaluation, span
from pyss3.server import Live_Test

from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.model_selection import train_test_split

## Mount Share

In [10]:
from google.colab import drive
from sys import path

drive.mount('/content/drive', force_remount=True)
path_root = '/content/drive/My Drive/NLP Lab'
path.append(path_root)
data_root = '/data'
single_label_dir = '/SB1_Train-attribute'
multi_label_dir = '/SB1_Train_Multilabel'

Mounted at /content/drive


## Load Data

In [12]:
text, labels = Dataset.load_from_files(path_root + data_root + single_label_dir, folder_label=False)

Loading 'PRICES' documents: 100%|██████████| 11/11 [00:00<00:00, 351.42it/s]


In [0]:
from util import load_multi_label
multi_text, multi_labels = load_multi_label(path_root + data_root + multi_label_dir + '/attribute-text.txt', path_root + data_root + multi_label_dir + '/attribute-labels.txt')

In [28]:
print("Length Single Label:", len(text), len(labels))
print("Length Multi Label:", len(multi_text), len(multi_labels))

Length Single Label: 4914 4914
Length Multi Label: 3747 3747


In [0]:
X_train_single, X_val_single, y_train_single, y_val_single = train_test_split(text, labels, test_size=0.2)
X_train_multi, X_val_multi, y_train_multi, y_val_multi = train_test_split(multi_text, multi_labels, test_size=0.2)

## Classification

In [0]:
def binarizer(y_pred, Classes):
  binary_label = [0]*len(Classes)
  indices = [Classes.index(label) for label in y_pred]
  for index in indices:
    binary_label[index] = 1
  return binary_label

### Training on Singlelabel

In [55]:
clf = SS3(a=1)
clf.train(X_train_single, y_train_single)

Training on 'USABILITY': 100%|██████████| 11/11 [00:00<00:00, 29.44it/s]


### Evaluation on Multilabel

In [56]:
loss = 0
count = 0
classes_low = [label.lower() for label in set(labels)]
classes_up = [label.upper() for label in set(labels)]
for index in range(len(X_val_multi)):
  count += 1
  y_pred = clf.classify_multilabel(X_val_multi[index])
  binary_y_pred = binarizer(y_pred, classes_low)
  binary_y_true = binarizer(multi_labels[index], classes_up)
  cur_loss = hamming_loss(binary_y_true, binary_y_pred)
  if count <= 3:
    print("predicted:", y_pred)
    print("actual:", multi_labels[index])
    print(cur_loss)
    print("-------------")
  loss += cur_loss
print("...")
print("..")
print(".")
print("Final Loss:", loss/count)

predicted: ['general']
actual: ['GENERAL']
0.0
-------------
predicted: ['general']
actual: ['OPERATION_PERFORMANCE']
0.18181818181818182
-------------
predicted: ['general']
actual: ['OPERATION_PERFORMANCE', 'GENERAL']
0.09090909090909091
-------------
...
..
.
Final Loss: 0.14727272727272794
